# Выбор локации для скважины

От руководства добывающей компании «ГлавРосГосНефть» поступила задача определения места, где бурить новую скважину.

Были предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. **Цель** - построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Также необходимо проанализировать возможную прибыль и риски техникой *Bootstrap.*

Для достижения цели необходимо выполнить следующие шаги для выбора локации:

- в избранном регионе найти месторождения, для каждого определить значения признаков;
- построить модель и оценить объём запасов;
- выбрать месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- найти суммарную прибыль отобранных месторождений.

Для анализа были предоставлены три файла, содержащих данные геологоразведки трёх регионов: geo_data_0.csv, geo_data_1.csv и geo_data_2.csv.

# Содержание
* [Загрузка и подготовка данных](#Загрузка-и-подготовка-данных)
* [Обучение и провека модели](#Обучение-и-проверка-модели)
* [Подготовка к расчету прибыли](#Подготовка-к-расчету-прибыли)
* [Расчет прибыли и рисков](#Расчет-прибыли-и-рисков)

## Загрузка и подготовка данных

In [1]:
# загрузим все необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from scipy import stats as st
from numpy.random import RandomState
pd.options.mode.chained_assignment = None

In [2]:
# загрузим три таблицы с данными по каждому региону
df_1 = pd.read_csv('/datasets/geo_data_0.csv')
df_2 = pd.read_csv('/datasets/geo_data_1.csv')
df_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
# посмотрим на первые 5 строчек первой таблицы
df_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
# посмотрим на первые 5 строчек второй таблицы
df_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
# посмотрим на первые 5 строчек третьей таблицы
df_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Согласно приложенной документации:
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

In [6]:
# посмотрим на общую информацию о первой таблице
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
# посмотрим на общую информацию о второй таблице
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
# посмотрим на общую информацию о третьей таблице
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Три таблицы имеют одинаковую размерность без пропусков. Проверим данные на наличие дубликатов.

In [9]:
# проверка на наличие дубликатов
print("Количество дубликатов в первой таблице:", df_1.duplicated().sum())
print("Количество дубликатов во второй таблице:", df_2.duplicated().sum())
print("Количество дубликатов в третьей таблице:", df_3.duplicated().sum())

Количество дубликатов в первой таблице: 0
Количество дубликатов во второй таблице: 0
Количество дубликатов в третьей таблице: 0


**Вывод**

На первом этапе были рассмотрены три таблицы с данными георазведки в разных регионах - df_1, df_2 и df_3. Таблицы имеют одинаковую размерность (5 столбцов и 100000 строк). 1 столбец - id - имеет тип данных object, остальные - float. Дубликатов не обнаружено, как и пропусков.

## Обучение и проверка модели

На данном этапе разобьем данные на обучающую и валидационную выборки. Затем обучим модель и сделаем предсказания на валидационной выборке. После чего найдем средний запас предсказанного сырья и RMSE модели. Поскольку данные операции необходимо будет сделать для каждого региона, то необходимо будет составить функции.

In [10]:
# выделим признаки и целевой признак в каждом регионе
features_1 = df_1.drop(columns=['id', 'product'], axis=1)
target_1 = df_1['product']
features_2 = df_2.drop(columns=['id', 'product'], axis=1)
target_2 = df_2['product']
features_3 = df_3.drop(columns=['id', 'product'], axis=1)
target_3 = df_3['product']

In [11]:
# построим функцию для разбивки каждой выборки на обучающую и валидационную, а также сразу стандартизируем количественные
# признаки
def test_split(df, features, target):
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25,
                                                                                   random_state=12345)
    scaler = StandardScaler()
    scaler.fit(features_train)
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    return features_train, features_valid, target_train, target_valid

In [12]:
# воспользуемся ранее составленной функцией для таблиц df_1, df_2 и df_3
features_train_1, features_valid_1, target_train_1, target_valid_1 = test_split(df_1, features_1, target_1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = test_split(df_2, features_2, target_2)
features_train_3, features_valid_3, target_train_3, target_valid_3 = test_split(df_3, features_3, target_3)

In [13]:
# построим функцию для расчета RMSE и среднего запаса предсказанного сырья
def ml(df, features_train, target_train, features_valid, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    predictions = pd.Series(predictions)
    # расчет средней квадратической ошибки
    RMSE = mean_squared_error(target_valid, predictions)**0.5
    # расчет среднего запаса предсказанного сырья
    mean_product_predict = predictions.mean()
   
    print('RMSE:', RMSE)
    print('Средний запас предсказанного сырья:', mean_product_predict)
    
    return (predictions, RMSE, mean_product_predict)

In [14]:
# применим функцию ml для расчета RMSE и среднего запаса предсказанного сырья в трех регионах, а также для создания 
# переменной с предсказаниями модели
print('Первый регион')
predictions_1, RMSE_1, mean_product_predict_1 = ml(df_1, features_train_1, target_train_1, features_valid_1, target_valid_1)
print('-----------------------------------------------------')
print('Второй регион')
predictions_2, RMSE_2, mean_product_predict_2 = ml(df_2, features_train_2, target_train_2, features_valid_2, target_valid_2)
print('-----------------------------------------------------')
print('Третий регион')
predictions_3, RMSE_3, mean_product_predict_3 = ml(df_3, features_train_3, target_train_3, features_valid_3, target_valid_3)

Первый регион
RMSE: 37.5794217150813
Средний запас предсказанного сырья: 92.59256778438008
-----------------------------------------------------
Второй регион
RMSE: 0.8930992867756167
Средний запас предсказанного сырья: 68.72854689544579
-----------------------------------------------------
Третий регион
RMSE: 40.02970873393434
Средний запас предсказанного сырья: 94.96504596800506


**Вывод**

Модель линейной регрессии показала разные результаты в трех регионах. Среднее квадратическое отклонение в первом и третьем регионах составило 37.579 и 40.029 соответственно, а во втором регионе - всего 0.893, что говорит о ненадежности предсказаний модели в регионах с высоким RMSE. При этом средний запас предсказанного сырья во втором регионе самый низкий, однако этому значению можно доверять с большей уверенностью.

## Подготовка к расчёту прибыли

In [15]:
# сохраним все ключевые значения для расчетов в отдельных переменных
N = 500 # количество исследуемых точек в регионе
M = 200 # количество лучших точек, выбираемых с помощью машинного обучения
BUDGET = 10000000000 # бюджет на разработку скважин в регионе
INCOME_PER_PROD = 450000 # доход с каждой единицы продукта (1000 бар.)
LOSS_PROBABILITY = 0.025 # вероятность убытков

In [16]:
# рассчитаем достаточный объем сырья для безубыточной разработки новой скважины
average_budget_for_oil = BUDGET / M # средний бюджет на разработку одной скважины в регионе
necessary_product = average_budget_for_oil / INCOME_PER_PROD # достаточный объем сырья для безубыточной разработки
print('Достаточный объем сырья для безубыточной разработки новой скважины', necessary_product, 'тыс. баррелей')

Достаточный объем сырья для безубыточной разработки новой скважины 111.11111111111111 тыс. баррелей


Найдем среднее значение запасов сырья в каждом регионе и сравним его с безубыточным значением.

In [17]:
def comparison(df):
    if df['product'].mean() > necessary_product:
        print('В регионе в среднем достаточно сырья для безубыточной разработки новой скважины.')
    else:
        print('В регионе в среднем недостаточно сырья для безубыточной разработки новой скважины.')

In [18]:
# 1-ый регион
comparison(df_1)

В регионе в среднем недостаточно сырья для безубыточной разработки новой скважины.


In [19]:
# 2-ый регион
comparison(df_2)

В регионе в среднем недостаточно сырья для безубыточной разработки новой скважины.


In [20]:
# 3-ый регион
comparison(df_3)

В регионе в среднем недостаточно сырья для безубыточной разработки новой скважины.


На данном этапе все необходимые для расчет прибыли показатели были сохранены в переменные. Также было установлено, что в регионах в среднем недостаточно сырья для безубыточной разработки новой скважины.

## Расчёт прибыли и рисков 

In [21]:
# построим функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели
def income(predictions, target):
    target = pd.Series(target).reset_index(drop=True)
    predictions = pd.Series(predictions).reset_index(drop=True)
    top_pred = predictions.sort_values(ascending=False)
    top_target = target[top_pred.index][:200]
    revenue = top_target.sum() * INCOME_PER_PROD 
    return round((revenue - BUDGET) / 1000000000, 3)

In [22]:
print('Первый регион')
print('Прибыль:', income(predictions_1, target_valid_1), 'млрд. руб.')
print('-----------------------------------------------------')
print('Второй регион')
print('Прибыль:', income(predictions_2, target_valid_2), 'млрд. руб.')
print('-----------------------------------------------------')
print('Третий регион')
print('Прибыль:', income(predictions_3, target_valid_3), 'млрд. руб.')

Первый регион
Прибыль: 3.321 млрд. руб.
-----------------------------------------------------
Второй регион
Прибыль: 2.415 млрд. руб.
-----------------------------------------------------
Третий регион
Прибыль: 2.71 млрд. руб.


Посчитаем риски с помощью процедуры Bootstrap. Для этого построим функцию.

In [23]:
def bootstrap(predictions, target):
    state = np.random.RandomState(12345)
    target = pd.Series(target).reset_index(drop=True)
    predictions = pd.Series(predictions).reset_index(drop=True)
    revenue =[]
    
    for i in range(1000):
        target_sample = target.sample(n=500, replace=True, random_state=state)
        predictions_sample = predictions[target_sample.index]
        revenue.append(income(predictions_sample, target_sample))
    revenue = pd.Series(revenue)
    lower = revenue.quantile(0.025)
    higher = revenue.quantile(0.975)
    mean = revenue.mean()
    risk = (revenue < 0).mean()
    
    print('2.5-% квантиль:', round(lower, 3), 'млрд. руб.')
    print('97.5-% квантиль:', round(higher, 3), 'млрд. руб.')
    print('Средняя прибыль:', round(mean, 3), 'млрд. руб.')
    print('Средние убытки:', round(risk, 3), 'млрд. руб.')

In [24]:
print('Первый регион')
bootstrap(predictions_1, target_valid_1)
print('-----------------------------------------------------')
print('Второй регион')
bootstrap(predictions_2, target_valid_2)
print('-----------------------------------------------------')
print('Третий регион')
bootstrap(predictions_3, target_valid_3)

Первый регион
2.5-% квантиль: -0.111 млрд. руб.
97.5-% квантиль: 0.91 млрд. руб.
Средняя прибыль: 0.396 млрд. руб.
Средние убытки: 0.069 млрд. руб.
-----------------------------------------------------
Второй регион
2.5-% квантиль: 0.034 млрд. руб.
97.5-% квантиль: 0.852 млрд. руб.
Средняя прибыль: 0.456 млрд. руб.
Средние убытки: 0.015 млрд. руб.
-----------------------------------------------------
Третий регион
2.5-% квантиль: -0.163 млрд. руб.
97.5-% квантиль: 0.95 млрд. руб.
Средняя прибыль: 0.404 млрд. руб.
Средние убытки: 0.076 млрд. руб.


**Вывод**

В процессе исследования было установлено, что второй регион принесет самую высокую прибыль от добычи нефти на новых скважинах (0.456 млрд. руб.). При этом риск убытков в данном регионе самый низкий (0.015 млрд. руб.). Также в нем самые высокие значения 95-% доверительного интервала